<a href="https://colab.research.google.com/github/Nocimaa/Binance_Bot/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv"

--2025-09-17 14:45:43--  https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28620838 (27M) [text/plain]
Saving to: ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’

btc_1h_adx_2017-08- 100%[===================>]  27.29M  --.-KB/s    in 0.09s   

2025-09-17 14:45:44 (303 MB/s) - ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’ saved [28620838/28620838]



In [2]:
import pandas as pd
import numpy as np

In [25]:
base = pd.read_csv("btc_1h_adx_2017-08-17_to_2025-09-17(in).csv")

df = base[base["timestamp"] < "2025-09-01 00:00:00+02:00"]
predict_df = base[base["timestamp"] >= "2025-09-01 00:00:00+02:00"]


def preprocessing(df, index_label=True):
  if index_label:
    df["label"] = np.log(df["open"].shift(-1) / df["open"])
  df = df.drop("timestamp", axis=1)


  features = ["open","high","low","close","volume","ATR14","MACD"]
  window = 3
  for f in features:
      for w in range(1, window+1):
          df[f"{f}_lag{w}"] = df[f].shift(w)


  df.dropna(inplace=True)
  return df

df = preprocessing(df)
predict_df = preprocessing(predict_df, index_label=False)

/tmp/ipython-input-4179757494.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = np.log(df["open"].shift(-1) / df["open"])


In [7]:
df.head()

,open,high,low,close,volume,ADX,+DI,-DI,ATR14,BB_width,...,close_lag3,volume_lag1,volume_lag2,volume_lag3,ATR14_lag1,ATR14_lag2,ATR14_lag3,MACD_lag1,MACD_lag2,MACD_lag3
3,4066.00,4088.12,4066.00,4088.10,5.090504,19.814435,15.006875,21.950910,56.892466,0.125607,...,4043.63,3.904708,54.912335,49.206310,59.567272,60.976292,62.194469,-19.635683,-21.766400,-20.330822
4,4063.00,4063.00,4000.01,4023.11,14.970767,20.869256,13.409755,27.587343,59.120862,0.127199,...,4036.30,5.090504,3.904708,54.912335,56.892466,59.567272,60.976292,-16.617763,-19.635683,-21.766400
5,4023.11,4023.11,4000.00,4000.00,6.477680,21.848881,13.018320,26.783322,56.548657,0.127527,...,4074.00,14.970767,5.090504,3.904708,59.120862,56.892466,59.567272,-19.248307,-16.617763,-19.635683
6,4000.00,4023.11,3934.43,3951.49,22.207985,23.613933,11.616981,31.859436,58.843753,0.127666,...,4088.10,6.477680,14.970767,5.090504,56.548657,59.120862,56.892466,-22.933453,-19.248307,-16.617763
7,3914.59,4023.11,3911.79,3989.17,7.695301,25.496193,10.141237,30.395799,62.592057,0.123878,...,4023.11,22.207985,6.477680,14.970767,58.843753,56.548657,59.120862,-29.429070,-22.933453,-19.248307


In [8]:
y = df["label"]
X = df.drop("label", axis=1)

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

k = 20
selector = SelectKBest(score_func=f_regression, k=k)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled  = y_scaler.transform(y_test.values.reshape(-1,1))



X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return self.fc4(x)

model = RegressionNet(input_dim=k)
model = model.to(device)

In [13]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
batch_size = 256

In [14]:
for epoch in range(epochs):
    permutation = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Évaluation train
    with torch.no_grad():
        preds = model(X_train_tensor)
        mse = criterion(preds, y_train_tensor).item()
    print(f"Epoch {epoch+1}/{epochs}, Train MSE={mse:.4f}")

Epoch 1/50, Train MSE=0.0243
Epoch 2/50, Train MSE=0.0199
Epoch 3/50, Train MSE=0.0179
Epoch 4/50, Train MSE=0.0185
Epoch 5/50, Train MSE=0.0139
Epoch 6/50, Train MSE=0.0141
Epoch 7/50, Train MSE=0.0187
Epoch 8/50, Train MSE=0.0134
Epoch 9/50, Train MSE=0.0128
Epoch 10/50, Train MSE=0.0142
Epoch 11/50, Train MSE=0.0121
Epoch 12/50, Train MSE=0.0134
Epoch 13/50, Train MSE=0.0153
Epoch 14/50, Train MSE=0.0132
Epoch 15/50, Train MSE=0.0119
Epoch 16/50, Train MSE=0.0111
Epoch 17/50, Train MSE=0.0111
Epoch 18/50, Train MSE=0.0117
Epoch 19/50, Train MSE=0.0122
Epoch 20/50, Train MSE=0.0112
Epoch 21/50, Train MSE=0.0126
Epoch 22/50, Train MSE=0.0123
Epoch 23/50, Train MSE=0.0118
Epoch 24/50, Train MSE=0.0111
Epoch 25/50, Train MSE=0.0107
Epoch 26/50, Train MSE=0.0130
Epoch 27/50, Train MSE=0.0100
Epoch 28/50, Train MSE=0.0124
Epoch 29/50, Train MSE=0.0100
Epoch 30/50, Train MSE=0.0096
Epoch 31/50, Train MSE=0.0106
Epoch 32/50, Train MSE=0.0098
Epoch 33/50, Train MSE=0.0098
Epoch 34/50, Train 

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import torch

with torch.no_grad():
    # prédictions sur GPU
    preds = model(X_test_tensor)

    # ramener sur CPU avant conversion en numpy
    preds_cpu = preds.cpu().numpy()
    y_true_cpu = y_test_tensor.cpu().numpy()

    # si tu as utilisé un scaler pour y
    preds_orig = y_scaler.inverse_transform(preds_cpu)
    y_true_orig = y_scaler.inverse_transform(y_true_cpu)

    # calcul métriques
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    import numpy as np

    mse_test = mean_squared_error(y_true_orig, preds_orig)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_true_orig, preds_orig)

print(f"\nTest MSE (original scale) = {mse_test:.4f}")
print(f"Test RMSE (original scale) = {rmse_test:.4f}")
print(f"Test MAE (original scale)  = {mae_test:.4f}")




Test MSE (original scale) = 0.0000
Test RMSE (original scale) = 0.0007
Test MAE (original scale)  = 0.0003


/tmp/ipython-input-4014891763.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = np.log(df["open"].shift(-1) / df["open"])


ValueError: could not determine the shape of object type 'DataFrame'

In [43]:
def predict_new_row(new_row, model, selector, scaler_X, scaler_y, feature_columns, device='cpu'):
    """
    Prédit le label à partir d'une nouvelle ligne de données.

    Args:
        new_row (list ou dict): Les valeurs de la nouvelle observation
        model (torch.nn.Module): modèle PyTorch déjà entraîné
        scaler_X (StandardScaler): scaler pour X
        scaler_y (StandardScaler): scaler pour y
        feature_columns (list): noms des colonnes/features utilisées par le modèle
        device (str): 'cpu' ou 'cuda'

    Returns:
        float: prédiction en échelle originale
    """

    # 1️⃣ Créer DataFrame
    if isinstance(new_row, list):
        df_new = pd.DataFrame([new_row], columns=feature_columns)
    elif isinstance(new_row, dict):
        df_new = pd.DataFrame([new_row])
    else:
        df_new = new_row
    # 2️⃣ Supprimer colonne timestamp si présente
    if "timestamp" in df_new.columns:
        df_new = df_new.drop(columns=["timestamp"])

    # 3️⃣ Appliquer la même sélection de features si nécessaire


    X_new = df_new.values

    X_new = selector.transform(X_new)
    # 4️⃣ Normaliser
    X_new_scaled = scaler_X.transform(X_new)

    # 5️⃣ Convertir en tensor et envoyer sur device
    X_tensor = torch.tensor(X_new_scaled, dtype=torch.float32).to(device)

    # 6️⃣ Prédiction
    model.eval()
    with torch.no_grad():
        pred_scaled = model(X_tensor)
        # Dénormaliser
        pred = scaler_y.inverse_transform(pred_scaled.cpu().numpy())

    return float(pred[0][0])


variation = predict_new_row(list(predict_df.iloc[2].drop("label")), model, selector, scaler, y_scaler, X.columns, device)

In [44]:
float(predict_df.iloc[0]["open"])

108222.37

In [45]:
float(predict_df.iloc[3]["open"]) * (1 - variation)

108933.38095300586

In [87]:
t = 0
wallet = 10
lever = 5
# fee_rate = 0.0004

for i in range(len(predict_df) - 1):
  variation = predict_new_row(list(predict_df.iloc[i].drop("label")), model, selector, scaler, y_scaler, X.columns, device)
  if abs(variation) > 0.001:
    # print(i, variation)
    t += 1


    str_variation = "baisse" if variation < 0 else "monte"

    # print(f"[{str_variation}] Ouverture : {float(predict_df.iloc[i]['open'])}\n Prediction : {float(predict_df.iloc[i]['open']) * (1 + variation)}\n Reel : {predict_df.iloc[i + 1]['open']}")
    price_open = float(predict_df.iloc[i]['open'])
    price_next = float(predict_df.iloc[i+1]['open'])

    if variation > 0:
      position = wallet * lever          # montant total de la position
      borrowed = wallet * (lever - 1)    # montant emprunté
      units = position / price_open      # nombre d’unités achetées
      # cost_fee = position * fee_rate     # frais achat
      proceeds = units * price_next      # valeur à la vente
      # sell_fee = proceeds * fee_rate     # frais vente
      wallet = (proceeds - borrowed)  # gain net
    elif variation < 0:
      position = wallet * lever
      borrowed = wallet * (lever - 1)
      units = position / price_open  # unités empruntées
      close_position = price_next * units
      open_position = price_open * units



      wallet = (open_position + open_position - close_position - borrowed)


    print(f"Solde : {wallet}")

    # print(float(predict_df.iloc[i]["open"]))
    # print(float(predict_df.iloc[i + 1]['open']))
    # print(float(predict_df.iloc[i]["open"]) * (1 + variation))
print(t)

Solde : 10.131349685400608
Solde : 10.714691031670448
Solde : 10.89250254062977
Solde : 10.969298857169171
Solde : 11.126591359501276
Solde : 11.299911970958483
Solde : 11.858584998885327
Solde : 12.21288784788468
Solde : 12.65912099845309
Solde : 12.773309558874416
Solde : 12.932998829807289
Solde : 13.484922567524222
Solde : 13.546796911717138
Solde : 13.649618475160551
Solde : 13.737865124327811
Solde : 13.865081214940318
Solde : 14.18746076551193
Solde : 14.793399389598946
Solde : 16.39380355101673
Solde : 16.54867817960414
Solde : 16.943106175512057
Solde : 17.041107341159716
Solde : 17.2466349322212
Solde : 17.37097365432804
Solde : 17.851063766037825
Solde : 18.082504294115623
Solde : 18.188611066499647
Solde : 18.32179112609697
Solde : 18.482527341558566
Solde : 18.746392751832943
Solde : 19.029317684154847
Solde : 19.35230771513021
Solde : 19.607058504697704
Solde : 19.728351685345288
Solde : 20.016714649201035
Solde : 20.34375712682865
Solde : 21.061231058726875
Solde : 21.40